In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [7]:
from torch.utils.data import DataLoader, Dataset

In [8]:
import pandas as pd
import string

In [9]:
from torchvision import transforms
from PIL import Image

In [10]:
from pathlib import Path

In [11]:
import torchvision.transforms.functional as TF

In [54]:
def remove_spaces(str_):
    return ' '.join(str_.split())

In [69]:
class ImageCaptioningDataset(Dataset):
    def __init__(self,caption_file,image_dir,image_transform=transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])):
        self.image_transform = image_transform
        self.image_dir = Path(image_dir)
        captions_csv = pd.read_csv(caption_file, sep="|")
        self.captions_csv = captions_csv.rename(lambda x: x.strip(), axis=1)
        self.captions_csv.comment.loc[19999] = self.captions_csv.comment_number.loc[19999][3:]
        self.captions_csv.comment_number.loc[19999] = 4
        self.captions_csv['comment'] = self.captions_csv['comment'].str.lower()
        self.captions_csv['comment'] = self.captions_csv['comment'].str.strip()
        self.captions_csv['comment'] = self.captions_csv['comment'].apply(remove_spaces)
        self.captions_csv['comment'] = self.captions_csv['comment'].str.translate(str.maketrans('', '', string.punctuation))

        display(self.captions_csv['comment'].loc[0])
        self.tokenizer = get_tokenizer('spacy', language='en_core_web_sm',)
        self.list_of_tokens = self.captions_csv['comment'].apply(self.tokenizer)
        self.vocab = build_vocab_from_iterator(self.list_of_tokens)
        self.vocab.append_token('<START>')
        self.vocab.append_token('<END>')
        self.vocab.append_token('<PAD>')
        self.list_of_tokens = list(map(lambda x: ['<START>'] + x + ['<END>'], self.list_of_tokens))
    def __len__(self):
        return self.captions_csv.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.image_dir / Path(self.captions_csv.loc[idx,'image_name'])
        image = Image.open(image_path)
        image = TF.to_tensor(image)
        if self.image_transform:
            image = self.image_transform(image)
        return image, self.vocab(self.list_of_tokens[idx]), len(self.list_of_tokens[idx])